# 1) Import texts and preprocess 

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_excel('ecb_pressconferences.xlsx', header = 0, na_values = 'NaN')
data2 = pd.read_excel('ecb_interviews.xlsx', header=0, na_values = 'NaN')
#data = pd.merge_ordered(data1, data2)
#data = data.sort_values('date', ascending=False)
type(data.text[0])
# data.text[0].count(' ')
data.text[0]


# Example text, latest pressconference in 2017 before preprocessing

In [ ]:
aaa = data.text.tolist()

nr = range(217)
bbb =[]
for x in nr:
    ccc = aaa[x].count(" ")
    
    bbb.append(ccc)
    
# create DataFrame
datum = data.date
ddd = pd.DataFrame(bbb, index = datum)

ddd.plot(kind='line', figsize=(20,10))



In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
import string


# preprocessor tokenizes texts first, then strips punctiation, numbers and stopwords (meaningless words) and converts
# entire text to lowercase, Part-of-Speech-tags words to lemmatize subsequently, stem tokens and finally filter for stopwwords
# again. Tokens are joint as a sting and returned.


def preprocessor(clean):
    
    tokens = word_tokenize(clean.lower()) # tokenize text and transform to lowercase
  
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens] # stripped = list
    words = [word for word in stripped if word.isalpha()] # words = list 

    stop_words = stopwords.words('english')
#     stop_words = set(stopwords.words('english'))
#     finterms = pd.read_excel('Finterms_wordlist.xlsx') # include finterm dictionary by Loughran & McDonald (2011)
#     for word in set(finterms.get('lm wordlist')):
#         stop_words.add(word)
    stop_words = [w.lower() for w in stop_words]

    words_cleaned = [w for w in words if not w in stop_words] # tokens in lowercase that are stripped for punctuation

    def get_wordnet_pos(treebank_tag):

        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return None # for easy if-statement 
    
    lemmatizer = WordNetLemmatizer()
    tagged = nltk.pos_tag(words_cleaned)
    lemmatized = []
    
    for word, tag in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:# not supply tag in case of None
            lemma = lemmatizer.lemmatize(word) 
        else:
            lemma = lemmatizer.lemmatize(word, pos=wntag)
        lemmatized.append(lemma)

    ps = PorterStemmer()
    stemmed = []
    for w in lemmatized:
        stemmed.append(ps.stem(w))

    words_preprocessed = [w for w in stemmed if not w in stop_words]
     
    text_preprocessed = " ".join(words_preprocessed)
   
    return text_preprocessed


def dataframe_preprocessor(frame):
   
    for cell in ['text']:
        frame[cell] = frame[cell].apply(preprocessor)
    
    return frame

dataframe_preprocessed = dataframe_preprocessor(data)
dataframe_preprocessed.head()


In [ ]:
dataframe_preprocessednew = dataframe_preprocessed
dataframe_preprocessednew['nr of communications']= 1

df_preprnew = dataframe_preprocessednew.drop(columns=['headline','text','type of publication'])
df_preprnew['date']=pd.to_datetime(df_preprnew['date'])
df_preprnew=df_preprnew.set_index('date')

df_preprnew = df_preprnew.resample('Q').sum().sort_index(ascending = False)


df_preprnew.plot(kind='bar', yticks=[2,3,4,5,6,7],figsize=(20,10))


# 2) Build corpus and create term-frequency-inverse-document-frequency matrix

In [ ]:
corpus = dataframe_preprocessed.text.tolist()
print(type(corpus)) 
print(type(corpus[0]))
print(len(corpus)) 
print(len(corpus[0]))
print(corpus[216].count(" "))
corpus[0]


# Output shows again the latest pressconference in 2017 but this time preprocessed.

# Explain corpus, picture it eg. how many words exist across corpus, most frequent words, distribution (what words in which text?
# how did central bank press conferences develop over time?
# -> number of publications per quarter, word lenght, etc.

In [ ]:
nr = range(217)
acac=[]
for x in nr:
    ababab = corpus[x].count(" ")
    
    acac.append(ababab)
    
# create DataFrame
datum = dataframe_preprocessed.date
abab = pd.DataFrame(acac, index = datum)

abab.plot(kind = 'line', figsize=(20,10))


In [ ]:
# development stopwords

nr_stop = ddd.sub(abab)
nr_stopperc = nr_stop.divide(ddd)
nr_stopperc.plot(kind = 'line', figsize=(20,10))

nr_stopperc.describe()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus) # tfidf = sparse csr-matrix
print(tfidf)



# tfidf matrix indicates importance of every term for document and corpus in order to distinguish documents across corpus 
# and themes across documents

In [ ]:
words = tfidf_vectorizer.get_feature_names()

# get 25 most important terms of corpus:
feature_array = np.array(words)
tfidf_sorting = np.argsort(tfidf.toarray()).flatten()[::-1]
top_25 = feature_array[tfidf_sorting][:25]
print("Most important 25 terms: {}".format(top_25))

# get 25 most frequent words
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus)
words1 = vectorizer.get_feature_names()
feature_array1 = np.array(words1)
tf_sorting = np.argsort(tf.toarray()).flatten()[::-1]
frequent_25 = feature_array1[tf_sorting][:25]
print("Most frequent 25 terms: {}".format(frequent_25))


# Below the 25 most important words indicated by the tfidf matrix are shown as well as the 25 most frequent words

# 3) Derive topics from corpus via NMF

In [ ]:
from sklearn.decomposition import NMF
import numpy as np


# Non-negative Matrix Factorization (NMF) allows to classify texts and leads to similar outputs as probablistic Latent
# Semantic Analysis. Comparing outputs containing different quanities of topics derived led to best results of regression 
# if _____ (currently 15)  topics are to be derived


model = NMF(n_components=15)

W = nmf = model.fit_transform(tfidf)        # W = Tocpics to document
H = model.components_                       # H = Terms to topic, factorization matrix
                       
    
date = dataframe_preprocessed.date
index = pd.DatetimeIndex(date)


idx_to_word = np.array(tfidf_vectorizer.get_feature_names())

for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-15:]]])))
    



# Currently the attribution for each row in W is not a percentage, but we want to assign each document to any topic which it can be at least 10% attributed to
sums = np.sum(W, axis=1)
W_percent = W / sums[:, None]


df = pd.DataFrame(W_percent, index)
df.head()


# Output describes most important words for determined topic. Below the W matrix is shown. W represents the percentage,
# to what extend a text consists of a certian topic



In [ ]:
# If more than _____ (again comparing results to derive optimal value, currently 10% 'topic limit') percent of the text was
# devoted to a derived topic, it counted as mentioned. The code below counts the number of mentionings of the subsequent 
# topic per quarter.



#How many latent topics should we look for?
#Rundown of the challenges associated with trying to determine the number of topics to factorize into. Include nmf_similarity plot and give a rundown of the thinking behind it as well as the PCA scree plot that gives another approach to the same question.




def topic_selector(value):
    if value < 0.25:
        return 0
    else:
        return 1
        
        
def dataframe_topics(data, i):
    for cell in [i]:
        data[cell] = data[cell].apply(topic_selector)
  
    return data


ii = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i in ii:
    dfw = dataframe_topics(df,i)
# print(dfw.loc[:,dfw.any()]) #check if one topic has 0 mentions higher than 10% of text -> no, if columns are same amount

dfw = dfw.resample('Q',).sum().sort_index(ascending = False) # summarize mentions during quarter 
dfw.head()


In [ ]:
dfw.plot(kind='line', yticks=[0,1,2,3,4], colormap='gist_rainbow',figsize=(20,10))

In [ ]:
print(dfw.sum())
dfw.describe()



In [ ]:
# export matrix to excel file

writer = pd.ExcelWriter('AAAAA.xlsx')
dfw.to_excel(writer)
writer.save()

In [ ]:
# variables for visulaization

# H and W
sums = np.sum(H, axis=1)
H_percent = H / sums[:, None]

print(type(H_percent))
print(H_percent.shape)

print(type(W_percent))
print(W_percent.shape)

length = np.asarray(acac)
print(type(length))
print(length.shape)

print(type(feature_array1))
print(feature_array1.shape)

TF = tf.toarray().transpose()
TFF = TF.sum(axis=1)
print(type(TFF))
print(TFF.shape)


In [ ]:
import pyLDAvis

graph = pyLDAvis.prepare(H_percent,W_percent, length, feature_array1, TFF, R=15)
pyLDAvis.display(graph)

In [ ]:
# yearly summarized number of mentions quarterly, topics with no metions deleted
dfw_annually = pd.read_excel('dfw_P_15_0-25_agg-current.xlsx', sheet_name='an', index_col=0)


# 4) Bulid DataFrames for dependent and independent variables

In [ ]:
# 1) bulid DataFrame for independent variables

import statsmodels.api as sm

files = ['10_years_int_rate.xlsx','ea_debt.xlsx','euribor_int_rates.xlsx','exchange_rates.xlsx','gdp_growth.xlsx','m3_growth.xlsm','unemployment_rate.xlsx', 'time.xlsx']

dataframes = [pd.ExcelFile(f) for f in files]

intr = dataframes[0].parse('annually_aggregated')
debt = dataframes[1].parse('annually_aggregated')
euribor = dataframes[2].parse('annually_aggregated')
exch = dataframes[3].parse('annually_aggregated')
gdp = dataframes[4].parse('annually_aggregated')
m3 = dataframes[5].parse('annually_aggregated')
unempl = dataframes[6].parse('annually_aggregated')
time = dataframes[7].parse('Tabelle1')

frames = [intr] 
frames.append(euribor.delta_euriday_an)
frames.append(exch.delta_dollar_an) 
frames.append(gdp.delta_gdp_an) 
frames.append(m3.delta_m3_an) 
frames.append(time.fincri)
frames.append(time.year) 

# frames.append(debt.delta_debt_an)
# frames.append(euribor.delta_eurimon_an)
# frames.append(euribor.delta_euri3mon_an) 
# frames.append(euribor.delta_euri6mon_an)
# frames.append(euribor.delta_euri12mon_an) 
# frames.append(exch.delta_pound_an) 
# frames.append(exch.delta_franc_an) 
# frames.append(exch.delta_yen_an) 
# frames.append(unempl.delta_unempl_an)
# frames.append(time.qter) 



new_df = pd.concat(frames, axis=1)


ndf = new_df.set_index('quarter')
ndf = ndf.resample('Q').sum().sort_index(ascending = False)


masterframe = pd.concat([dfw_annually, ndf], axis = 1)
masterframe


# Prossible control variables are the quarterly data on annually aggregated changes in:

#    - 10 years interest rate (StatisticalDataWarehouse: IRS.M.U2.L.L40.CI.0000.EUR.N.Z) 
#    - Government debt (Eurostat: Quarterly government debt [gov_10q_ggdebt])
#    - euribor rates (Eurostat: Geldmarktzinssätze - Vierteljährliche Daten [irt_st_q])
#    - exchange rates [Pund Sterling, Swiss Franc, Yen, USD] (Eurostat: Euro/Ecu-Wechselkurse - Vierteljährliche Daten [ert_bil_eur_q])
#    - GDP (StatisticalDataWarehouse: MNA.Q.Y.I8.W2.S1.S1.B.B1GQ._Z._Z._Z.EUR.LR.GY)
#    - Money aggregate M3 (StatisticalDataWarehouse: BSI.Q.U2.N.V.M30.X.I.U2.2300.Z01.A)
#    - Unemployment rate (Eurostat: Unemployment by sex and age - quarterly average [une_rt_q])

#    - variable for year
#    - variable for quarter
#    - dummy for financial crisis 07/08

# due to strong correlations within the variables, most are excluded as shown in the next section.
# The ouput contains the mentions of the derived topics along with the selected controll variables. 




In [ ]:
masterframe_judg = masterframe.drop(masterframe.index[0:4])
masterframe_judg = masterframe_judg.reset_index()
masterframe_judg = masterframe_judg.drop('index', axis=1)
masterframe_judg

ndf_judg = ndf.drop(ndf.index[0:4])
ndf_judg = ndf_judg.reset_index()
ndf_judg = ndf_judg.drop('quarter', axis=1)
ndf_judg

In [ ]:
# 2) bulid Dataframe for dependent variable


spf_errs = pd.read_excel('SPF_errors.xlsx', sheet_name = 'data')
spf_errs = spf_errs.set_index('quarter')
spf_errs = spf_errs.resample('Q').sum().sort_index(ascending = False)
spf_errs = spf_errs[:68]
spf_errs24 = spf_errs.err_24_month[:72]


# ABSOLUTE VALUES
spf_errs = spf_errs.abs()
spf_errs





# DataFrame shows the quarterly inflation forecast error of SPF 12 and 24 month rolling forecast.
# The HICP inflation is taken from the ECBs StatisticalDataWarehouse (ICP.M.U2.N.000000.3.ANR),
# as well as the SPF 12 month rolling forecast (SPF.M.U2.HICP.POINT.P12M.Q.AVG) and the 24 month rolling forecast
# (SPF.M.U2.HICP.POINT.P24M.Q.AVG)

In [ ]:
spf_errs_judg = spf_errs.reset_index()
spf_errs_judg = spf_errs_judg.drop('quarter', axis=1)
spf_errs_judg

# 5) Correlation

In [ ]:
# correlation among topics
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,12))
mask = np.zeros_like(dfw_annually.corr())
mask[np.triu_indices_from(mask)] = True
sns.heatmap(data=dfw_annually.corr(),mask=mask,vmin=-1, vmax=1, square=True, cmap="YlGnBu",linewidths=.5, ax=ax)
dfw_annually.corr()

In [ ]:
# correlation among control variables

mask = np.zeros_like(ndf.corr())
mask[np.triu_indices_from(mask)] = True
print(sns.heatmap(data=ndf.corr(), mask=mask, vmin=-1, vmax=1, square=True, cmap="YlGnBu",linewidths=.5))
ndf.corr()

# correlations among control variables (>0,5):

# intr, euri6.        0,50                       # euriday, eurimon:   0,99
# intr, euri12:       0,57                       # euriday, euri3mon:  0,98
                                                 # euriday, euri6mon:  0,96
# debt, euriday:      0.65                       # euriday, euri12mon: 0,92
# debt, eurimon:      0,63                       # euriday, gdp:       0,84
# debt, euri3mon:     0,63                       # euriday, unempl:   -0,77
# debt, euri6mon:     0,63                       # eurimon, euri3:     0,99
# debt, euri12mon:    0,57                       # eurimon, euri6:     0,98
# debt, gdp:         -0,71                       # eurimon, euri12:    0,95
# debt, unempl:       0,79                       # eurimon, gdp:       0,86
                                                 # eurimon, unempl:   -0,78
# franc, yen:         0,56                       # eurimon3, euri6:    0,99
# franc, dollar:      0,55                       # eurimon3, euri12:   0,97
# yen, dollar:        0,62                       # eurimon3, gdp:      0,87
                                                 # eurimon3, unempl:  -0,78
# gdp, unempl:       -0,88                       # euri6, euri12:      0,99
                                                 # euri6, gdp:         0,86
# m3, ficri:          0,62                       # euri6, unempl:     -0,77
                                                 # euri12, gdp:        0,84
                                                 # euri12, unempl:    -0,75
        

In [ ]:
# correlation between topics and controll variables

mask = np.zeros_like(masterframe.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(14,13))
sns.heatmap(data=masterframe.corr(), mask=mask, vmin=-1, vmax=1, square=True, cmap="YlGnBu",linewidths=.9, ax=ax)

masterframe.corr()


# 6) Run regression

In [ ]:
# OLS 12 month rolling forecast error
masterframe_judg = sm.add_constant(masterframe_judg)
model = sm.OLS(spf_errs_judg.err_12_month, masterframe_judg)
result = model.fit()
print(result.summary())